## データセット作成
このNotebookでは、Kaggleで公開されている花の画像を利用し、
ABEJA PlatformでImage Classificationを利用するためのデータセットを作成します。

### Step 1: Load data
データセットを作成するためのデータをダウンロードします。
このステップではKaggleで提供されている花のデータセットをダウンロードして利用します。

#### Data Download

In [13]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
file_id = '1w6Fn_ZjLlPzAswSRFj6uuusYod87apHJ'
destination = './flowers-recognition.zip'
download_file_from_google_drive(file_id, destination)
print('Download OK')

#### ダウンロードした画像データを解凍・ファイルチェック
ダウンロードしたデータをunzipし、ファイル数、フォルダ名をチェックします。

In [ ]:
# unzip download data
n_files = !unzip -l ./flowers-recognition.zip | grep .jpg | wc -l
!unzip -o ./flowers-recognition.zip | pv -l -s {n_files[0]} > /dev/null

In [ ]:
from glob import glob

# load filenames for images
file_names = list(glob('./flowers/*/*'))
dir_names = list(glob('./flowers/*'))

# print number of images in dataset
print('There are %d total images.' % len(file_names))

In [ ]:
# select directories
selected_dirnames = [d for d in dir_names]
print(selected_dirnames)

### Step 2: DataLakeにデータをアップロードします。
このステップでは、ダウンロードしたデータにメタデータ(Label)を付与してDataLakeにアップロードします。
※事前にDataLakeチャンネルの作成が必要です。

In [ ]:
# set credential
credential = {
    'user_id': 'user-XXXXXXXXXXXXX',
    'personal_access_token': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
}

organization_id='XXXXXXXXXXXXX'

In [ ]:
from abeja.datalake import Client as DatalakeClient

# set datalake channel_id
channel_id = 'XXXXXXXXXXXXX'

datalake_client = DatalakeClient(organization_id, credential)
channel = datalake_client.get_channel(channel_id)

import os
from tqdm import tqdm

# upload directory data to datalake
for d in tqdm(selected_dirnames):
    # convert to uppercase and remove numbers
    label_name = os.path.basename(d)
    metadata = {'label': label_name}
    channel.upload_dir(d, metadata=metadata)

### Step 3: データレイクに保存したファイルのメタデータ(Label)を利用してJSONを出力
データセット作成に必要な「プロパティ」に入力するJSONをメタデータを利用して出力します。
※出力されたJSONの最初と最後のカンマ以外をコピーして利用ください。

In [ ]:
import json

labels = sorted([os.path.basename(d) for d in selected_dirnames])
labels_and_id = []
label_to_id = {}

for i, name in enumerate(labels):
    label_to_id[name] = i
    labels_and_id.append({'label_id': i,
                   'label': name})
    
# define category name
category_name = 'flower-classificaiton'

# create dataset label
category = {
    'category_id': 0,
    'name': category_name,
    'labels': labels_and_id}

props = {'categories': [category]}
json.dumps(props)

### Step 4: Datalakeに保存したファイルからデータセットを作成します。
このステップでは、DataLakeに保存したファイルを利用してデータセットを作成します。
※ラベルについては、メタデータ(Label)で設定されている情報を利用します。

In [ ]:
# create dataset by importing datalake files
from abeja.datasets import Client as DatasetClient

dataset_client = DatasetClient(organization_id, credential)

# define dataset id
dataset_id = 'XXXXXXXXXXXXX'

dataset = dataset_client.get_dataset(dataset_id)

for f in tqdm(channel.list_files()):
    data_uri = f.uri
    filename = f.metadata['filename']
    label = f.metadata['label']
    label_id = label_to_id[label]
    
    if os.path.splitext(filename)[1].lower() == '.jpg' or \
    os.path.splitext(filename)[1].lower() == '.jpeg':
        content_type = 'image/jpeg'
    elif os.path.splitext(filename)[1].lower() == '.png':
        content_type = 'image/png'
    else:
        print('{} is invalid file type.'.format(filename))
        continue
    
    source_data = [{'data_uri': data_uri, 'data_type': content_type}]
    attributes = {'classification': [{'category_id': 0, 'label_id': label_id, 'label': label}]}
    dataset.dataset_items.create(source_data, attributes)